[Source](https://www.philschmid.de/fine-tune-flan-t5)

[Youtube](https://www.youtube.com/watch?v=msgLLudzlLg)

[Github Repo](https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/flan-t5-samsum-summarization.ipynb)

[Dataset](https://huggingface.co/datasets/ChanceFocus/flare-edtsum)

# Setup Development Environment

In [ ]:
!pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr --upgrade
# install git-fls for pushing model and logs to the hugging
# !sudo apt-get install git-lfs --yes
!pip install peft
!pip install -q -U bitsandbytes scipy einops accelerate trl
!pip install evaluate

  Using cached peft-0.10.0-py3-none-any.whl (199 kB)
  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.

In [ ]:
import evaluate

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
from datasets import load_dataset
from huggingface_hub import notebook_login
from random import randrange
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import concatenate_datasets
# import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
import evaluate
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import pipeline
from transformers import BitsAndBytesConfig
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, PeftConfig

In [ ]:
notebook_login()

# Evaluation before fine tuning

In [ ]:
!pip install evaluate
!pip install datasets
!pip install rouge_score

In [9]:
# Import the load function from the evaluate module
from evaluate import load
from datasets import load_dataset
from tqdm.auto import tqdm
import torch

In [4]:
dataset = load_dataset("ChanceFocus/flare-edtsum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
# dataset
dataset = dataset["test"].train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'query', 'answer', 'text'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['id', 'query', 'answer', 'text'],
        num_rows: 400
    })
})

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
# tokenizer = T5Tokenizer.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
# Loading the 'rouge' metric from the library
rouge = load('rouge')

# Define your predictions and references
# predictions = ["Your summary 1", "Your summary 2"]
# references = ["Reference summary 1", "Reference summary 2"]

# Compute the scores
# results = rouge.compute(predictions=predictions, references=dataset['test']['answer'])

# Print the scores
# print(results)

In [17]:
evaluate_on_test(model,tokenizer)

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

{'rouge1': 0.31938072529560046, 'rouge2': 0.16657099807833367, 'rougeL': 0.27259095592115623, 'rougeLsum': 0.2727968581534159}


In [13]:
max_source_length = 512
def classify(texts_to_classify: str,model,tokenizer):
    """Classify a batch of texts using the model."""
    inputs = tokenizer(
        texts_to_classify,
        padding="max_length",
        truncation=True,
        max_length=max_source_length,
        return_tensors="pt",
    )
    # inputs = inputs.to("cuda" if torch.cuda.is_available() else "cpu")

    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=150,
            num_beams=2,
            early_stopping=True,
        )

    predictions = [
        tokenizer.decode(output, skip_special_tokens=True) for output in outputs
    ]
    return predictions

In [14]:
def evaluate_on_test(model,tokenizer):
    """Evaluate the model on the test dataset."""
    predictions_list, labels_list = [], []

    batch_size = 16  # Adjust batch size based GPU capacity
    num_batches = len(dataset['test']) // batch_size + (
        0 if len(dataset['test']) % batch_size == 0 else 1
    )
    progress_bar = tqdm(total=num_batches, desc="Evaluating")

    for i in range(0, len(dataset['test']), batch_size):
        batch_texts = dataset['test']["query"][i : i + batch_size]
        batch_labels = dataset['test']["answer"][i : i + batch_size]

        batch_predictions = classify(batch_texts,model,tokenizer)

        predictions_list.extend(batch_predictions)
        labels_list.extend([str(label) for label in batch_labels])

        progress_bar.update(1)

    progress_bar.close()
    # report = classification_report(labels_list, predictions_list)
    results = rouge.compute(predictions = predictions_list , references = labels_list)
    print(results)

# Load and prepare EDTSUM dataset

In [ ]:
# {
#   "id": "13818513",
#   "summary": "Amanda baked cookies and will bring Jerry some tomorrow.",
#   "dialogue": "Amanda: I baked cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"
# }

In [ ]:
dataset_id = "ChanceFocus/flare-edtsum"

In [ ]:
# Load dataset from the hub
dataset = load_dataset(dataset_id)


# Train dataset size: 14732
# Test dataset size: 819

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

DatasetDict({
    test: Dataset({
        features: ['id', 'query', 'answer', 'text'],
        num_rows: 2000
    })
})

In [ ]:
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

In [ ]:
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"Text: \n{sample['text']}\n---------------")
print(f"Answer: \n{sample['answer']}\n---------------")

# Load Tokenizer

In [ ]:
model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = T5Tokenizer.from_pretrained(model_id)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = dataset.map(lambda x: tokenizer(x["query"], truncation=True), batched=True, remove_columns=["query", "answer","text","id"])
max_source_length = max([len(x) for x in tokenized_inputs['test']["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = dataset.map(lambda x: tokenizer(x["answer"], truncation=True), batched=True, remove_columns=["query", "answer","text","id"])
max_target_length = max([len(x) for x in tokenized_targets['test']["input_ids"]])
print(f"Max target length: {max_target_length}")

Max source length: 512
Max target length: 303


# Preprocess dataset

In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["query"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["answer"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["query", "answer","text","id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['test'].features)}")

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
tokenized_dataset = tokenized_dataset["test"].train_test_split(test_size=0.2)

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 400
    })
})

# Prepare and preprocess the model for PeFT training

In [ ]:
!pip install accelerate

In [ ]:
import accelerate

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

model = T5ForConditionalGeneration.from_pretrained(model_id,device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True,)

model.config.use_cache = False

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# from peft import prepare_model_for_kbit_training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v", "k", "o"], # you have to know the target modules, it varies from model to model
    lora_dropout=0.05,
    bias="none",
    # task_type="CAUSAL_LM"
    task_type="SEQ_2_SEQ_LM"
)

# SEQ_CLS = "SEQ_CLS"
#     SEQ_2_SEQ_LM = "SEQ_2_SEQ_LM"
#     CAUSAL_LM = "CAUSAL_LM"
#     TOKEN_CLS = "TOKEN_CLS"
#     QUESTION_ANS = "QUESTION_ANS"
#     FEATURE_EXTRACTION = "FEATURE_EXTRACTION"


model = get_peft_model(model, config) # Wrap the base model with get_peft_model() to get a trainable PeftModel

# Trainable parameters

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 1769472 | total: 249347328 | Percentage: 0.7096%


# Evaluation Code

We are going to use evaluate library to evaluate the rogue score.

In [ ]:
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:


# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

# Training

In [ ]:

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=10,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    # hub_model_id=repository_id,
    # hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.111911,40.549600,23.707100,36.978600,37.065300,17.635000
2,No log,2.075336,40.441400,24.114300,37.110700,37.165300,17.427500
3,2.340500,2.056396,40.749600,24.147500,37.425300,37.448200,17.660000
4,2.340500,2.040511,41.137000,24.319700,37.841200,37.901000,17.507500
5,2.202100,2.029091,41.565300,24.665200,38.295300,38.326500,17.500000
6,2.202100,2.022660,41.436800,24.492500,38.077800,38.101400,17.482500
7,2.202100,2.019122,41.194600,24.280700,37.819300,37.880700,17.560000
8,2.175400,2.012609,41.478200,24.370800,38.037500,38.080000,17.547500
9,2.175400,2.010934,41.565900,24.573600,38.200500,38.339900,17.470000
10,2.153000,2.010632,41.616500,24.603800,38.246900,38.386100,17.465000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation

TrainOutput(global_step=2000, training_loss=2.2177579345703125, metrics={'train_runtime': 1807.6301, 'train_samples_per_second': 8.851, 'train_steps_per_second': 1.106, 'total_flos': 1.1043090726912e+16, 'train_loss': 2.2177579345703125, 'epoch': 10.0})

# Evaluation Results

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 2.010632038116455,
 'eval_rouge1': 41.6165,
 'eval_rouge2': 24.6038,
 'eval_rougeL': 38.2469,
 'eval_rougeLsum': 38.3861,
 'eval_gen_len': 17.465,
 'eval_runtime': 74.9834,
 'eval_samples_per_second': 5.335,
 'eval_steps_per_second': 0.667,
 'epoch': 10.0}

# Save the adapter and Merge it with the base model(for saving within Colab)

One thing to keep in mind is that you can’t merge the 8 bit/4 bit base model with Lora (as of right now) so you have to reload the model with full precision.

In [ ]:
trainer.save_model("FlanT5_fin_summarization_finetuned")

In [ ]:
#To merge base model with fine tuned model
repo_id = "google/flan-t5-base"
use_ram_optimized_load=False

base_model = T5ForConditionalGeneration.from_pretrained(
    repo_id,
    device_map='auto',
    trust_remote_code=True,
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


In [ ]:

base_model.config.use_cache = False

# Load Lora adapter
model = PeftModel.from_pretrained(
    base_model,
    "FlanT5_fin_summarization_finetuned",
    )



In [ ]:
merged_model = model.merge_and_unload()

# Save the model

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

from huggingface_hub import login
login()

In [ ]:
# Save our tokenizer and create model card
repository_id = "steve1989/FlanT5_financial_summarization_finetuned"
# tokenizer.save_pretrained(repository_id)
# trainer.create_model_card()
# Push the results to the hub
merged_model.push_to_hub(repository_id)
tokenizer.push_to_hub(repository_id)

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/steve1989/FlanT5_financial_summarization_finetuned/commit/6f69b03260cea9ed967ad1e0de3eeab0ed0c8f7a', commit_message='Upload tokenizer', commit_description='', oid='6f69b03260cea9ed967ad1e0de3eeab0ed0c8f7a', pr_url=None, pr_revision=None, pr_num=None)

# Run Inference and summarize ChatGPT dialogues

In [ ]:
# load model and tokenizer from huggingface hub with pipeline
summarizer = pipeline("summarization", model="steve1989/FlanT5-financial-summarization-finetuned", device=0)

# select a random test sample
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")

# summarize dialogue
res = summarizer(sample["dialogue"])

print(f"flan-t5-base summary:\n{res[0]['summary_text']}")